In [1]:
import h5py
import numpy as np
import math
import pandas as pd

In [36]:
voxel_size = 3
max_x_part = 4
max_y_part = 4

In [37]:
def get_2D_data_from_h5(h5_path, part_name, Slice_name):
    with h5py.File(h5_path,'r') as h5:
        X_Axis = h5[part_name][Slice_name]['X_Axis']
        Y_Axis = h5[part_name][Slice_name]['Y_Axis']
        Area = h5[part_name][Slice_name]['Area']

        help_arr = np.column_stack((X_Axis, Y_Axis, Area))
        df = pd.DataFrame(help_arr, columns=['x','y','Area'])
        return df

In [38]:
df = get_2D_data_from_h5("/home/jan/Documents/Voxel_Erstellung/example_hdf_3.hdf",'part_1','Slice_1')

In [39]:
#Berechnung der Anzahl an Voxeln (hier simpler Fall)
def get_number_voxel (max_x_part, max_y_part, voxel_size):
    n_voxels_x = math.ceil(int(max_x_part)/voxel_size)
    n_voxels_y = math.ceil(int(max_y_part)/voxel_size)
    return n_voxels_x, n_voxels_y #tuple is returned 

In [40]:
num_voxels = get_number_voxel (max_x_part, max_y_part, voxel_size)
num_voxels

(2, 2)

In [46]:
def fill_2D_voxel_area (voxel_size, num_voxels_x, num_voxels_y, df, filling_method):
    array_area = np.zeros([voxel_size,voxel_size]) #creating an empty array of dimensions voxel_size*voxel_size
    for i in range(voxel_size*num_voxels_x, voxel_size*(num_voxels_x+1)): #iterating over x
        for j in range(voxel_size*num_voxels_y,voxel_size*(num_voxels_y+1)): #iterating over y

            if df[(df['x'] == i) & (df['y'] == j)].shape[0] == 1: #here subset of the original dataframe is created an filtrered --> shape[0] returns the number of rows of this df subset  
                #finding the area value for a certain point in the part-data-dataframe and allocating it to a position in the array
                area_i = df.loc[(df['x'] == i) & (df['y'] == j)]
                array_area[i-num_voxels_x*voxel_size][j-num_voxels_y*voxel_size] = area_i['Area']
                
            elif df[(df['x'] == i) & (df['y'] == j)].shape[0] > 1:
                #if there are more values than just one the maximum value is used for the voxel-datapoint; other methods of dealing with multiple values need to be considered 
                array_area[i-num_voxels_x*voxel_size][j-num_voxels_y*voxel_size] = df[(df['x'] == i) & (df['y'] == j)]['area'].max()

            
            elif df[(df['x'] == i) & (df['y'] == j)].shape[0] == 0 and filling_method == 'Zeros':
                array_area[i-num_voxels_x*voxel_size][j-num_voxels_y*voxel_size] = 0
                    
            #elif filling_method == 'Mean': #with this method all the missing datapoints are getting filled with the mean of the non-missing datapoints 
             #   array_area[i][j] =      

    return array_area


In [47]:
voxel_hdf = h5py.File("voxels_over_border.hdf5", "w")
voxel_hdf.close()

In [48]:
for n_vox_y_init in range(num_voxels[0]): #iterating over number of voxels in y-direction
    for n_vox_x_init in range(num_voxels[1]):#iterating over number of voxels in x-direction 
        
        array_area = fill_2D_voxel_area(voxel_size, n_vox_x_init, n_vox_y_init, df,'Zeros')
                
        with h5py.File("voxels_over_border.hdf5", "a") as voxel_hdf:
            #creating a voxel with the numbers of voxels in both direction in its name and filling it with data
            voxel_hdf.create_group('voxel_{}_{}'.format(n_vox_x_init,n_vox_y_init))
            voxel_hdf['voxel_{}_{}'.format(n_vox_x_init,n_vox_y_init)].create_dataset('X-Axis',data = np.repeat(np.arange(0,voxel_size,1),voxel_size))
            voxel_hdf['voxel_{}_{}'.format(n_vox_x_init,n_vox_y_init)].create_dataset('Y-Axis',data = np.tile(np.arange(0,voxel_size,1),voxel_size))
            voxel_hdf['voxel_{}_{}'.format(n_vox_x_init,n_vox_y_init)].create_dataset('Area', data = array_area.flatten())
        #print(array)       